# Create LULC

In [1]:
%matplotlib inline
%load_ext chime

import os
from glob import glob
import pickle
import chime

import rasterio as rio
import gdal
import numpy as np
import pandas as pd
import geopandas as gpd
from skimage.morphology import binary_closing, binary_opening, square, remove_small_holes, disk, square

from skimage.filters.rank import modal, mean_bilateral
from datetime import datetime, timedelta, time
from time import sleep
import nbimporter
#from KeyFunctions import *
import chime
from utils import *


def throttleProcessors(workDayStart, workDayEnd):
    startPause = time(*(map(int, workDayStart.split(':'))))
    endPause = time(*(map(int, workDayEnd.split(':'))))
    nowTime = datetime.today().time()
    if nowTime < endPause or nowTime > startPause:
        print(f"Current time is {nowTime}. Setting processor/thread use to 4")
        return 4
    else:
        return 8

def wait_start(workDayStart="8:30", workDayEnd="18:00", force=False):
    startPause = time(*(map(int, workDayStart.split(':'))))
    endPause = time(*(map(int, workDayEnd.split(':'))))
    nowTime = datetime.today().time()
    if nowTime < endPause and nowTime > startPause and datetime.today().isoweekday() <= 5 and not force:
        waitTimeSeconds = datetime.combine(datetime.today(), endPause) - datetime.combine(datetime.today(), nowTime)
    else:
        waitTimeSeconds = False
    return waitTimeSeconds


def createClassifiedRaster(classification_model, ortho_file, classifiedFiles_loc, overwrite=False):
    start = datetime.now()
    day = datetime.now().strftime("%Y%m%d")
    daynum = 1
    output_image = os.path.join(classifiedFiles_loc, os.path.basename(ortho_file).replace(".tif", f"_classLGBNewishGBLM.tif"))#_{day}-{daynum}.tif"))
    if os.path.exists(output_image) and not overwrite:
        print("File exists")
        return output_image
    try:
        with rio.open(ortho_file) as src:
            kwargs = src.profile
            kwargs.update(
                dtype= rio.uint8,
                count= 1,
            )

            data = src.read()
            descs = list(src.descriptions)
    except:
        print(f"ERROR: Unable to open {os.path.basename(ortho_file)}. Skipping")
        return None
    
    kwargs.update(
        dtype= rio.uint8,
        count= 1,
    )
    
    feature_names = classification_model.feature_name_

    bands = {desc:data[ib] for ib, desc in enumerate(descs)}
    
    #This is a training dataset not created with the others, but may be in the classifier. Create and add
    if "RGBNmean" in feature_names:
        print("Adding RGBNmean")
        rgbnMean = np.nanmean(data[:4], axis=0).astype(data.dtype)
        bands["RGBNmean"] = rgbnMean

    features = {fn:bands[fn] for fn in feature_names}# if fn in descs}
    
    featureArrays = list(features.values())
            
    trainingFeatures = np.stack(featureArrays)

    #print("Input Data shape", data.shape)
    
    # read the image into the proper format, adding indices if necessary
    img_swp = np.moveaxis(trainingFeatures, 0, -1)
    img_flat = img_swp.reshape(-1, img_swp.shape[-1])
    
    #flatten bands along axis
    img_flat = img_swp.reshape(-1, img_swp.shape[-1])
    
    # remove no data values, store the indices for later use
    # a later cell makes the assumption that all bands have identical no-data value arrangements
    m = np.ma.masked_invalid(img_flat)
    to_predict = img_flat[~m.mask].reshape(-1, img_flat.shape[-1])
    
    # predict
    #print("Beginning prediction...", datetime.now())
    img_preds = classification_model.predict(to_predict)
    
    # add the prediction back to the valid pixels (using only the first band of the mask to decide on validity)
    # resize to the original image dimensions
    output = np.zeros(img_flat.shape[0])
    output[~m.mask[:,0]] = img_preds.flatten()
    output = output.reshape(*img_swp.shape[:-1])
    
    # create our final mask
    mask = (~m.mask[:,0]).reshape(*img_swp.shape[:-1])
    
    with rio.open(output_image, 'w', **kwargs) as dst: 

        colors = {
            1: (12,42,235, 255),
            2: (41, 210, 219,255),
            3: (255, 214, 117, 255),
            4: (171, 224, 85, 255),
            5: (12, 100, 1, 255),
            6: (0, 192, 32, 255),
            7: (62, 62, 62, 255),
            8: (160, 160, 160, 255),
            9: (160, 37, 6, 255)
        }
        
        # write to the final file
        dst.write(output.astype(rio.uint8), 1)
        #dst.write_mask(mask)
        
        dst.write_colormap(1, colors)
        
    return output_image

    print(f"Classified to {os.path.abspath(output_image)}. \nClassification took {datetime.now()-start}")
    %chime
    
    
def classifyTiles(df, forceStart=False, overwrite=False):
    print(f"\n\nStarting processing for {len(df)} tiles\n")
    for i, r in df.iterrows():
        #bestlgbm_model.set_params(n_jobs=throttleProcessors("8:30", "18:00"))
        waitTime = wait_start(force=forceStart)
        if waitTime:
            buildVRT(classifiedFiles_loc, "EPC_30cmOrthoSegmented_Classified.vrt")
            pause_until = datetime.now() + waitTime
            print(f"waiting {timedelta(seconds=waitTime.seconds)} until {pause_until.strftime('%H:%m')}")
            sleep(waitTime.seconds)
            
        print(f"Starting {r.path}_{r.row} @ {datetime.now()}")
        result = createClassifiedRaster(classification_model=bestlgbm_model,
                                        ortho_file=r.OrthoFile,
                                        classifiedFiles_loc=classifiedFiles_loc,
                                        overwrite=overwrite)
        
    return result

In [3]:
bestlgbm_model = pickle.load(open("./Models/lightGBM_20210614.sav", 'rb'))

classifiedFiles_loc = "../EPCExtent_30cm/Orthos_Segmented_Classifiedv3"
#propsDir = r"../EPCExtent_30cm/Orthos_Segmentedv2_properties"
orthosDir = r"../EPCExtent_30cm/Orthos_Segmentedv3"
os.makedirs(classifiedFiles_loc, exist_ok=True)

tindex = gpd.read_file("../EPCExtent_30cm/Ortho_5kSubIndex.gpkg")
to_process = gpd.read_file("../temp/ToProcess.gpkg")

tindex["OrthoFile"] = tindex.apply(lambda r: findFile(path=r.path, row=r.row, directory=orthosDir), axis=1)
#tindex["PropsFile"] = tindex.apply(lambda r: findFile(path=r.path, row=r.row, files=propsDir), axis=1)

# ignore tiles which don't have input variables created
tindex = tindex[(~pd.isnull(tindex.OrthoFile))]

#prioritize central tucson and work out from there
tindex["centroid"] = tindex.geometry.centroid
central_tile = tindex[(tindex.path == "W1004789") & (tindex.row == "W449850")]
central_point = central_tile.centroid.values[0]

tindex["DistToCenter"] = tindex.centroid.apply(lambda c: int(c.distance(central_point)))
tindex.sort_values(by="DistToCenter", inplace=True)

#filter out files that are completed
#lulc_done = glob("../EPCExtent_30cm/Orthos_Segmented_Classified/*.tif")
tindex["Done"]= tindex.apply(lambda r: findFile(path=r.path, row=r.row, directory=classifiedFiles_loc),axis=1)
#tindex = tindex[pd.isnull(tindex["Done"])]




    
#toProcess = tindex[tindex.geometry.intersects(to_process.unary_union)].reset_index()                                 
#classifyTiles(toProcess)
#finished()
#classifyTiles(tindex)
#finished()

print("\nFINISHED")

C:\Users\BenJames\anaconda3\envs\geospatial\lib\site-packages\geopandas\geodataframe.py:294: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for f in features_lst:



FINISHED


In [4]:
tt = tindex[(tindex.path == "W989789") & (tindex.row == "W439850")]

In [5]:
%%time
t = classifyTiles(tt, forceStart=True, overwrite=True)
finished()



Starting processing for 1 tiles

Starting W989789_W439850 @ 2021-06-14 12:28:38.667388
Wall time: 4min 5s
